Documentação do imap: https://pypi.org/project/imap-tools/

Import das bibliotecas necessarias

imap_tools: O imap_tools é uma biblioteca do Python criada para tornar a vida
de quem precisa automatizar e-mails muito mais fácil. O imap_tools funciona
como um tradutor amigável que transforma esses comandos complicados em
comandos simples que qualquer pessoa entende.

para que ele serve? O IMAP é o protocolo (a regra) que permite que você
entre no email, abra as caixas de email e leia as mensagens. O imap_
tools é a ferramenta para

    -> se conectar com servidores/provedores (Gmail, Outlook, Yahoo)
    -> Navegar entre pastas (Caixa de Entrada, Enviados, Spam)
    -> Filtrar emails por remetente, assunto ou data.
    -> Baixar Anexos
    -> Marcar como lido ou mover emails de uma pasta para a outra.


As peças principais do imap_tools:

-> MailBox:É a conexão. É onde realizamos o login e selecionamos pasta.

-> fetch: É o comando de "buscar". Ele vai até o servidor e traz os 
e-mails baseados nos filtros que especificamos.

-> AND/OR/NOT: São as regras de lógica. Servem para dizer: "Quero e-mails deste remetente E(AND) com este assunto".

In [ ]:
#imap_tools: Biblioteca que possui a classe MailBox e as funções
# de regras de lógica.

# MailBox: Classe que contém os métodos para acessar e manipular as
# informações do e-mail acessado.

# AND: Operador lógico que iremos utilizar para filtrar as mensagens e anexos.
from imap_tools import MailBox, AND

# Função da biblioteca dotenv que tem como objetivo carregra
# variáveis de ambientes no código.
from dotenv import load_dotenv

# Import da biblioteca que possibilita que o python se conecte ao nosso sistema de
# arquivos.
import os

Carregando as variáveis de ambiente

In [ ]:
# Função que irá carregar as variáveis de ambiente no sistema
load_dotenv()

# Função da biblioteca os que irá acessar os valores carregados 
# pelo load_dotenv
# Observação: O google não tem mais suporte para o acesso de apps menos
# seguros, dito isso, ao invés de utilizar a sua senha de email, você deve
# gerar uma senha de app especifica para esse sistema.
EMAIL = os.getenv('EMAIL')

SENHA = os.getenv('SENHA')

Criando o objeto email

In [ ]:
# Ira comter o email que queremos acessar
usuario = EMAIL

# Ira conter a senha de acesso do email (senha de app)
senha = SENHA

# MailBox: Instância da classe que recebe como argumento em seu construtor
# o endereço do servidor do gmail (que pode ser encontrada na documentação ou
# na internet).

# login: Método da classe MailBox que recebe como argumento o usuario (email)
# e a senha que serão utilizados para acessar o email.
meu_email = MailBox("imap.gmail.com").login(usuario, senha)

Visualizando a lista de pastas do email

In [ ]:
# Vamos percorrer os valores do método list que irá conter os names do
# atributo folder que contém as pastas presentes no email.
for folder in meu_email.folder.list():
    print(folder.name)

INBOX
Rascunhos
[Gmail]
[Gmail]/Com estrela
[Gmail]/E-mails enviados
[Gmail]/Importante
[Gmail]/Lixeira
[Gmail]/Rascunhos
[Gmail]/Spam
[Gmail]/Todos os e-mails


Filtrando mensagens do email.

In [ ]:
# Diferente da aula, eu tive que especificar a pasta que vou utilizar
# no filtro de mensagens.
meu_email.folder.set('[Gmail]/E-mails enviados')

# list: Função que ira transformar o retorno do método fetch em uma lista.
# fetch: Método da classe meu_email que irá filtrar as mensagens de acordo
# com o que especificamos na função de operador lógico (AND)

# AND: função que irá receber como argumento o filtro que especificara a busca
# do fetch no e-mail. Argumentos do AND: 
# from_: Ira filtrar os emails do remetente
# to: Ira filtrar os emails do destinatário.
# Basicamente, estamos dizendo que queremos todas as mensagens enviadas
# do remetente usuário para o email f143acad@cps.sp.gov.br
lista_emails = list(meu_email.fetch(AND(from_=f"{usuario}", to="f143acad@cps.sp.gov.br")))

# Ira mostrar a quantidade de itens da lista de emails (itens encontrados
# através do filtro)
print("Emails encontrados: ", len(lista_emails))

Emails encontrados:  9


Visualizando o resultado do filtro aplicado no email

In [ ]:
# Ira percorrer a lista de email que contém os valores do filtro
# aplicado no email
for email in lista_emails:
    
    # Vamos mostrar os valores dos atributos subject (assunto) e text
    # (conteúdo da mensagem). Observação: Comentei o trecho de print,
    # por que as mensagens possuem dados sensiveis.
    #print(email.subject)
    #print(email.text)
    
    # O pass garante que um loop ou uma função funcione mesmo sem
    # um conteudo definido dentro delas, é uma forma do python
    # ignorar um loop ou uma função inativa. O pass é reconhecido
    # pelo python como uma operação nula.
    pass

pegar um anexo de email

In [ ]:
# ira filtrar as mensagens enviadas pelo usuário. 
lista_emails_anexos = meu_email.fetch(AND(from_=f"{usuario}"))

# Ira percorrer a lista de emails retornada pelo fetch
for email in lista_emails_anexos:
    
    # O fetch retorna um conjunto de objetos (onde cada objeto é um email que
    # contém atributos como subject, text, etc), dito isso cada objeto email
    # possui um atributo attachments que contém uma lista de anexos enviados
    # ou recebidos pelos emails presentes no fetch. Após essa explicação,
    # vamos verificar se a lista de attachments do email possui algum anexo,
    # ou seja, se a lista possui um tamanho maior que 0.
    if len(email.attachments) > 0:
        
        # Se a lista não estiver vázia, vamos percorre-la com o objetivo
        # de acessar e baixar os anexos.
        for anexo in email.attachments:
            
            # Diferente da aula, eu precisei remover os espaços em branco
            # e converter as letras maiusculas para minusculas do atributo
            # filename.
            nome_arquivo = anexo.filename.lower().replace(" ", "")
            
            # Após o tratamento do nome, vamos verificar se há algum
            # arquivo com o filename do nome do arquivo
            if "categorizaçãodeprodutos" in nome_arquivo:
                
                # Atributo que ira conter os bytes (conjunto de valores
                # binários) do arquivo encontrado
                informacoes_anexo = anexo.payload
                
                
                # with: é utilizado para o que chamamos de gerenciamento de
                # contexto. Em termos simples, ele serve para garantir que
                # um recurso (como um arquivo ou uma conexão de banco de dados
                # ) seja aberto e, mais importante, fechado automaticamente,
                # mesmo que ocorra um erro durante o processo. Imagine que
                # abrir um arquivo é como entrar em uma sala trancada: você
                # precisa da chave para entrar e precisa trancar a porta
                # ao sair para não gastar memória. O with é o assistente que garante que a porta seja trancada, não importa o que alcança
                # lá dentro.
                
                # open: Função do python que serve para abrir arquivos. 
                # Basicamente, ele vai verificar se o arquivo ja existe
                # e caso ele não exista, ele irá criar o arquivo com o
                # nome especificado (categorizacao.xlsx) e os bytes 
                # que irão conter os dados da variável informacoes_anexo. 
                
                # arquivo_excel: variável que irá conter os valores da
                # função open 
                with open("Categorizacao.xlsx", "wb") as arquivo_excel:
                    
                    # Ira escrever no arquivo que será criado os bytes
                    # que contem os dados do arquivo encontrado.
                    arquivo_excel.write(informacoes_anexo)

                    # Impressão do nome do arquivo encontrado
                    print("Anexo ", nome_arquivo, "encontrado")
    
    else:
        
        # Mensagem que irá aparecer toda vez que ele não encontrar o anexo
        # em um dos objetos email
        print("Não há anexos para baixar")
                
                

Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexos para baixar
Não há anexo

In [ ]:
# Função da biblioteca os que mostra o caminho que os esta acessando no momento
print(os.getcwd())

c:\Users\caike\Documents\Curso-de-automa--o-com-python\Localizando emails e anexos
